# Parsing and Cleaning PHEME RNR Dataset Events

This notebook performs data-cleaning and aggrigation into thread-level data. It also provides a useful sanity check. Run all the cells in this notebook to generate thread-level CSV files in the `data/threads` directory.

In [1]:
# Load dependencies for this Jupyter Notebook
import pandas as pd
import numpy as np
import networkx as nx
from functools import reduce
from lib.util import fetch_tweets

## Parsing and Cleaning Data
This step takes the raw PHEME rumor dataset and saves it tabular format as CSV file. The original PHEME dataset consists of JSON files organized into directories by event and category (rumor or non-rumor). These three functions below parse the data, save it as a CSV file (if necessary), and load it into this notebook as a Pandas DataFrame from the "cached" CSV file.

In [2]:
gw = fetch_tweets("germanwings-crash")

##  Tweet Level Features

| Name/Column       | Description                   | Type   | Notes  |
|-------------------|-------------------------------|--------| ------ |
| is_rumor          | Was this classified as rumor  | "bool" (`int`) | *Classification done by journalists* |
| thread            | Source tweet id               | `str`  |                                                   |
| in_reply_tweet    | Tweet ID in reply to          | `str`  |                                                   |
| event             | Name of the PHEME event       | `str`  | Corresponds to event in the PHEME dataset         |
| tweet_id          | Unique ID for tweet           | `str`  | This field is the ID referenced in `in_reply_tweet`     |
| is_source_tweet   | Was this classified as rumor  | "bool" (`int`) |                                                   |
| in_reply_user     | User ID in reply to           | `str`  |                                                   |
| user_id           | Twitter User's ID             | `str`  | This field is the ID referenced in `in_reply_user` |
| tweet_length      | Number of characters in tweet | `int`  |                                                   |
| urls_count        | Number of URLS in tweet       | `int`  |                                                   |
| hashtags_count    | Number of hashtags in tweet   | `int`  |                                                   |
| retweet_count     | Times the tweet was retweeted | `int`  |                                                   |
| favorite_count    | Number of times favorited     | `int`  |                                                   |
| mentions_count    | Number of users mentioned     | `int`  |                                                   |
| is_truncated      | Is this tweet truncated       | "bool" (`int`) | Did User type > 140 characters. [See Tweet updates](https://developer.twitter.com/en/docs/tweets/tweet-updates) |
| created              | Datetime Tweet was created    | `datetime` | |
| has_smile_emoji      | Does Tweet contain ""?        | "bool" (`int`) | 😊 is the smile emoji |
| user.tweets_count    | User's tweet total, currently | `int`  | |
| user.verified        | Is Twitter user verified?     | "bool" (`int`) |                                                   |
| user.followers_count | Total number of followers  | `int` | |
| user.listed_count    | ?? | `int` | | 
| user.friends_count   | ?? | `int` | |
| user.time_zone       | Timezone of the user's Twitter account | `str` | |
| user.desc_length     | Length of the user's biographic description | `int` |
| user.has_bg_img      | Does user have a profile background image?  | "bool" (`int`) |
| user.default_pric    | Does the user have the default profile picture | "bool" (`int`) |
| user.created_at      | Date and time Twitter account was activated | `datetime` | |
| user.utc_dist        | TK | `int` | See [this blog post time and the Twitter API](https://zacharyst.com/2017/04/05/assigning-the-correct-time-to-a-twee) |

## Germanwings Crash

In [3]:
gw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4489 entries, 0 to 4488
Data columns (total 40 columns):
is_rumor                4489 non-null int64
thread                  4489 non-null object
in_reply_tweet          4489 non-null object
event                   4489 non-null object
tweet_id                4489 non-null object
is_source_tweet         4489 non-null int64
in_reply_user           4489 non-null object
user_id                 4489 non-null object
tweet_length            4489 non-null int64
symbol_count            4489 non-null int64
user_mentions           4489 non-null int64
urls_count              4489 non-null int64
media_count             4489 non-null int64
hashtags_count          4489 non-null int64
retweet_count           4489 non-null int64
favorite_count          4489 non-null int64
mentions_count          4489 non-null int64
is_truncated            4489 non-null int64
created                 4489 non-null float64
has_smile_emoji         4489 non-null int64
sensi

The `.head` method prints out the 5 first rows in the dataframe

In [4]:
gw.head()

,is_rumor,thread,in_reply_tweet,event,tweet_id,is_source_tweet,in_reply_user,user_id,tweet_length,symbol_count,...,user.notifications,user.friends_count,user.time_zone,user.has_bg_img,user.default_pic,user.created_at,user.location,user.profile_sbcolor,user.profile_bgcolor,user.utc_dist
0,0,580319983676313601,nan,germanwings-crash,580319983676313601,1,nan,8330472,98,0,...,0,74,Madrid,1,0,1.187705e+12,1,16777215,11453380,0.0
1,0,580319983676313601,5.803199836763136e+17,germanwings-crash,580322851850461184,0,8330472.0,2307392966,109,0,...,0,1782,NaN,1,1,1.390520e+12,1,12639981,12639981,NaN
2,0,580319983676313601,5.803228518504612e+17,germanwings-crash,580323127089082368,0,2307392966.0,2535310842,36,0,...,0,121,NaN,0,0,1.401464e+12,1,0,0,NaN
3,0,580319983676313601,5.803231270890824e+17,germanwings-crash,580325737619685377,0,2535310842.0,2535310842,72,0,...,0,121,NaN,0,0,1.401464e+12,1,0,0,NaN
4,0,580321203757387776,nan,germanwings-crash,580321203757387776,1,nan,92771309,85,0,...,0,1354,London,1,0,1.259248e+12,1,13421772,16777215,NaN


## Boolean Columns

The `describe` method will give summary information about each column in the dataframe. Each of these columns, except `is_truncated` should have two unique values.

Just for a sanity check. The cell below converts these boolean columns into value of type `bool` and describes them.

In [5]:
bool_columns = ["is_rumor", "is_source_tweet", "is_truncated", 
                "has_smile_emoji", "user.verified", "user.has_bg_img", 
                "user.default_pic", "sensitive", "has_place", "has_coords", "user.notifications"]

gw[bool_columns].astype(bool).describe(include="bool")

,is_rumor,is_source_tweet,is_truncated,has_smile_emoji,user.verified,user.has_bg_img,user.default_pic,sensitive,has_place,has_coords,user.notifications
count,4489,4489,4489,4489,4489,4489,4489,4489,4489,4489,4489
unique,2,2,1,2,2,2,2,2,2,2,1
top,True,False,False,False,False,True,False,False,False,False,False
freq,2494,4020,4489,4487,4109,3992,2827,4455,4228,4363,4489


Some columns in some PHEME events have only one unique value for all tweets. So we'll drop any where they exist.

In [6]:
for col in gw.columns:
    if len(gw[col].unique()) == 1:
        gw.drop(col, inplace=True, axis = 1)

In [7]:
gw.describe()

,is_rumor,is_source_tweet,tweet_length,user_mentions,urls_count,media_count,hashtags_count,retweet_count,favorite_count,mentions_count,...,user.handle_length,user.name_length,user.friends_count,user.has_bg_img,user.default_pic,user.created_at,user.location,user.profile_sbcolor,user.profile_bgcolor,user.utc_dist
count,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,...,4489.000000,4489.000000,4489.000000,4489.000000,4489.000000,4.489000e+03,4489.000000,4.489000e+03,4.489000e+03,2969.000000
mean,0.555580,0.104478,90.788372,1.506572,0.185787,0.127423,0.298062,25.290933,9.049677,1.506572,...,11.704834,10.434172,1576.522834,0.889285,0.370238,1.308283e+12,0.724215,1.117811e+07,9.096543e+06,3.905945
std,0.496957,0.305913,39.920378,1.020736,0.400827,0.334150,0.693938,163.969869,70.400399,1.020736,...,4.375173,2.663997,8865.247468,0.313814,0.482922,6.714802e+10,0.446959,5.620359e+06,5.702912e+06,3.112538
min,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,1.165596e+12,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,0.000000,0.000000,56.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,9.000000,8.000000,184.000000,1.000000,0.000000,1.248215e+12,0.000000,1.106124e+07,1.710879e+06,1.000000
50%,1.000000,0.000000,97.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,12.000000,10.000000,448.000000,1.000000,0.000000,1.301649e+12,1.000000,1.263998e+07,1.263998e+07,5.000000
75%,1.000000,0.000000,130.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,...,15.000000,12.000000,1210.000000,1.000000,1.000000,1.365812e+12,1.000000,1.565873e+07,1.263998e+07,6.000000
max,1.000000,1.000000,151.000000,12.000000,2.000000,2.000000,8.000000,4388.000000,2541.000000,12.000000,...,20.000000,15.000000,453460.000000,1.000000,1.000000,1.427287e+12,1.000000,1.677722e+07,1.677722e+07,12.000000


## Thread Level Features

* **Bold features** represent high performing features identified in C. Buntain and J. Golbeck, ["Automatically Identifying Fake News in Popular Twitter Threads"](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8118443&isnumber=8118402)
* Features that are normalized are normalized by thread length


| Name                | Description                               | Type    | Notes |
| ---                 | ---                                       | ---     | ----- |
| thread              | Tweet ID of the source tweet              | `str`   | |
| favorite_count      | Normalized favorite total                 | `float` | |
| retweet_count       | Normlaized retweet total                  | `float` | |
| **hashtags_count**  | Normlaized hashtag total                  | `float` | |
| urls_count          | URL total normalized by thread length     | `float`  | |
| user.tweets_count   | Total tweets by thread users              | `float` | |
| event               | Name of PHEME event                       | `str`  | |
| is_rumor            | Either rumor or nonrumor                  | `bool` | |
| thread_length       | Number of tweets in the thread            | `int`  | |
| user.has_bg_img     | Ratio of users who have bg image          | `float`| |
| user.default_pic    | Ratio of users with default profile pic   | `float`| |
| **has_smile_emoji** | Number of smile emojis in the thread      | `int`  | 😊 is the smile emoji |
| user.verified       | Count of verified users in the thread normalized by thread length     | `float`  | |
| **src.followers_count** | The number of followers of the original poster of the thread. | `int` | |
| src.listed_count    | TODO | `int` | |
| src.user_verified   | TODO | `int` | |
| src.tweets_total    | TODO | `int` | |
| reply_var           | The variance in the timestamps of responses to the source tweet | `float` |
| src_age             | Difference in src user's creation and tweet creation            | `int`   | Measured in seconds |
| time_to_first_resp  | The difference between tweet creation datetime and 1st reply    | `int`   | Measured in seconds |
| time_to_last_resp   | The difference between tweet creation datetime and last reply   | `int`   | Measured in seconds |

In [8]:

def agg_tweets_by_thread(df):

    # Returns the proportion of True/1 values in col
    funcs = [np.mean, sum, np.var]
    agg_props = {
        "favorite_count": funcs,
        "user_mentions": funcs,
        "media_count": funcs,
        "sensitive": funcs,
        "has_place": funcs,
        "has_coords": funcs,
        "retweet_count": funcs,
        "hashtags_count": funcs,
        "urls_count": funcs,
        "user.tweets_count": funcs,
        "is_rumor": max,
        "tweet_id": len,
        "user.has_bg_img": funcs,
        "user.default_pic": funcs,
        "has_smile_emoji": funcs,
        "user.verified": funcs,
        "user.name_length": funcs,
        "user.handle_length": funcs,
        "user.profile_sbcolor": funcs,
        "user.profile_bgcolor": funcs,
    }
    rename = {
        "tweet_id": "thread_length",
        "has_url":"url_proportion",
    }

    def g(x):
        # Add size of largest user-to-user conversation component in each thread        
        d = []
        thread_tweets = list(x["tweet_id"])
        G = nx.from_pandas_edgelist(df[df.tweet_id.isin(thread_tweets)], "user_id", "in_reply_user")
        Gc = max(nx.connected_component_subgraphs(G), key=len)
        d.append(nx.number_connected_components(G))
        d.append(nx.diameter(Gc))
        return pd.Series(d, index=["component_count", "largest_cc_diameter"])
    
    # Step 0: Build graph-based features
    graph = df.groupby("thread").apply(g)
    
    # Step 1: Build simple aggregate features
    agg = df.groupby("thread")\
        .agg(agg_props)\
        .rename(columns=rename)
    
    agg.columns = [ "_".join(x) for x in agg.columns.ravel() ]
    agg = agg.rename(columns={"is_rumor_max": "is_rumor", "thread_length_len": "thread_length"})
    
    # Step 2: Builds some features off the source tweet, which has tweet_id == thread            
    src = df[df["is_source_tweet"] == 1][["thread",
                                          "user.followers_count", 
                                          "user.listed_count",
                                          "user.verified",
                                          "created",
                                          "user.created_at",
                                          "user.tweets_count"]] \
                         .rename(columns={"user.followers_count": "src.followers_count",
                                          "user.listed_count": "src.listed_count",
                                          "user.verified": "src.user_verified",
                                          "user.created_at": "src.created_at",
                                          "user.tweets_count": "src.tweets_total"})
    
    # Step 3: Build features off of the reply tweets
    def f(x):
        d = []
        
        # Get various features from the distribution of times of reply tweet
        d.append(min(x["created"]))
        d.append(max(x["created"]))
        d.append(np.var(x["created"]))
                
        return pd.Series(d, index=["first_resp", "last_resp","resp_var"])
        
    replies = df[df["is_source_tweet"] == False] \
        .groupby("thread") \
        .apply(f)

    graph_features = df.groupby("thread").apply(g)
    
    dfs = [agg, src, replies, graph]
    thrd_data = reduce(lambda left, right: pd.merge(left,right, on="thread"), dfs)
    
    # Step 3: Add miscelaneous features
    # Remember timestamps increase as time progresses
    # src.created_at < created < first_resp < last_resp
    thrd_data["time_to_first_resp"] = thrd_data["first_resp"] - thrd_data["created"]
    thrd_data["time_to_last_resp"] = thrd_data["last_resp"] - thrd_data["created"]
    
    return thrd_data

In [9]:
gw_thrds = agg_tweets_by_thread(gw)
gw_thrds.columns

Index(['thread', 'favorite_count_mean', 'favorite_count_sum',
       'favorite_count_var', 'user_mentions_mean', 'user_mentions_sum',
       'user_mentions_var', 'media_count_mean', 'media_count_sum',
       'media_count_var', 'sensitive_mean', 'sensitive_sum', 'sensitive_var',
       'has_place_mean', 'has_place_sum', 'has_place_var', 'has_coords_mean',
       'has_coords_sum', 'has_coords_var', 'retweet_count_mean',
       'retweet_count_sum', 'retweet_count_var', 'hashtags_count_mean',
       'hashtags_count_sum', 'hashtags_count_var', 'urls_count_mean',
       'urls_count_sum', 'urls_count_var', 'user.tweets_count_mean',
       'user.tweets_count_sum', 'user.tweets_count_var', 'is_rumor',
       'thread_length', 'user.has_bg_img_mean', 'user.has_bg_img_sum',
       'user.has_bg_img_var', 'user.default_pic_mean', 'user.default_pic_sum',
       'user.default_pic_var', 'has_smile_emoji_mean', 'has_smile_emoji_sum',
       'has_smile_emoji_var', 'user.verified_mean', 'user.verified_sum

In [10]:
gw_thrds.describe()

,favorite_count_mean,favorite_count_sum,favorite_count_var,user_mentions_mean,user_mentions_sum,user_mentions_var,media_count_mean,media_count_sum,media_count_var,sensitive_mean,...,created,src.created_at,src.tweets_total,first_resp,last_resp,resp_var,component_count,largest_cc_diameter,time_to_first_resp,time_to_last_resp
count,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,...,4.050000e+02,4.050000e+02,405.000000,4.050000e+02,4.050000e+02,4.050000e+02,405.000000,405.000000,4.050000e+02,4.050000e+02
mean,8.710762,97.195062,3221.805729,1.297139,16.644444,0.637482,0.170852,1.362963,0.112973,0.008894,...,1.427252e+12,1.255751e+12,68351.703704,1.427254e+12,1.427293e+12,9.712302e+14,2.439506,2.565432,2.610360e+06,4.124093e+07
std,17.072722,231.006084,20027.225635,0.644183,21.245780,0.741634,0.231598,1.988609,0.136270,0.045504,...,9.293428e+07,6.512534e+10,67962.053233,9.503021e+07,1.357477e+08,4.148492e+15,0.855628,1.427728,2.243148e+07,8.499159e+07
min,0.166667,1.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.427193e+12,1.167702e+12,481.000000,1.427193e+12,1.427194e+12,0.000000e+00,1.000000,1.000000,4.000000e+03,4.800000e+04
25%,2.200000,13.000000,23.861905,0.900000,3.000000,0.285714,0.000000,0.000000,0.000000,0.000000,...,1.427196e+12,1.213374e+12,14441.000000,1.427197e+12,1.427202e+12,1.723590e+11,2.000000,2.000000,7.700000e+04,2.507000e+06
50%,4.222222,32.000000,107.000000,1.142857,10.000000,0.424242,0.071429,1.000000,0.047619,0.000000,...,1.427199e+12,1.241688e+12,49113.000000,1.427201e+12,1.427215e+12,6.242125e+12,2.000000,2.000000,1.650000e+05,8.068000e+06
75%,7.600000,73.000000,450.000000,1.571429,24.000000,0.666667,0.272727,2.000000,0.214286,0.000000,...,1.427328e+12,1.285848e+12,113480.000000,1.427328e+12,1.427371e+12,1.004180e+14,3.000000,3.000000,5.420000e+05,3.703800e+07
max,224.800000,2556.000000,258141.273333,6.166667,185.000000,8.285714,1.000000,13.000000,0.500000,0.500000,...,1.427487e+12,1.423252e+12,520062.000000,1.427625e+12,1.427919e+12,4.198625e+16,8.000000,11.000000,4.284190e+08,7.236080e+08


In [11]:
fn = "data/threads/germanwings-crash.csv"
gw_thrds.to_csv(fn, index=False)
"Wrote data to %s" % fn

'Wrote data to data/threads/germanwings-crash.csv'